In [1]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import log, dot
import pandas as pd

%load_ext lab_black
%load_ext watermark

# Arrhythmia

A logistic regression example.

Adapted from [Unit 7: arrhythmia.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/arrhythmia.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/arrhythmia.csv).

Associated lecture video: Unit 7 lesson 15

## Problem statement

Patients who undergo Coronary Artery Bypass Graft Surgery (CABG) have an approximate 19-40% chance of developing atrial fibrillation (AF). AF can lead to blood clots forming causing greater in-hospital mortality, strokes, and longer hospital stays. While this can be prevented with drugs, it is very expensive and sometimes dangerous if not warranted. Ideally, several risk factors which would indicate an increased risk of developing AF in this population could save lives and money by indicating which patients need pharmacological intervention. Researchers began collecting data from CABG patients during their hospital stay such as demographics like age and sex, as well as heart rate, cholesterol, operation time, etc.. Then, the researchers recorded which patients developed AF during their hospital stay. Researchers now want to find those pieces of data which indicate high risk of AF. In the past, indicators like age, hypertension, and body surface area (BSA) have been good indicators, though these alone have not produced a satisfactory solution.

Fibrillation occurs when the heart muscle begins a quivering motion instead of a normal, healthy pumping rhythm. Fibrillation can affect the atrium (atrial fibrillation) or the ventricle (ventricular  fibrillation); ventricular fibrillation is imminently life threatening.

Atrial fibrillation is the quivering, chaotic motion in the upper chambers of the heart, known as the atria. Atrial fibrillation is often due to serious underlying medical conditions, and should be evaluated by a physician. It is not typically a medical emergency.

Ventricular fibrillation occurs in the ventricles (lower chambers) of the heart; it is always a medical emergency. If left untreated, ventricular fibrillation (VF, or V-fib) can lead to death within minutes. When a heart goes into V-fib, effective pumping of the blood stops. V-fib is considered a form of cardiac arrest, and an individual suffering from it will not survive unless cardiopulmonary resuscitation (CPR) and defibrillation are provided immediately.

DATA Arrhythmia
- Y = Fibrillation
- X1 = Age
- X2 = Aortic Cross Clamp Time
- X3 = Cardiopulmonary Bypass Time:
    - Bypass of the heart and lungs as, for example, in open heart surgery. Blood returning to the heart is diverted through a heart-lung machine (a pump-oxygenator) before returning it to the arterial circulation. The machine does the work both of the heart (pump blood) and the lungs (supply oxygen to red blood cells).
- X4 = ICU Time	(Intensive Care Unit)
- X5 = Avg Heart Rate	
- X6 = Left Ventricle Ejection Fraction
- X7 = Hypertension
- X8 = Gender [1 -Female; 0-Male]
- X9 = Diabetis
- X10 = Previous MI

In [2]:
data_df = pd.read_csv("../data/arrhythmia.csv")
data_df.info()
X = data_df.iloc[:, 1:].to_numpy()
# add intercept column to X
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
y = data_df["Fibrillation"].to_numpy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Fibrillation                   81 non-null     float64
 1   Age                            81 non-null     float64
 2   AorticCrossClampTime           81 non-null     float64
 3   CardiopulmonaryBypassTime      81 non-null     float64
 4   ICUTime                        81 non-null     float64
 5   AvgHeartRate                   81 non-null     float64
 6   LeftVentricleEjectionFraction  81 non-null     float64
 7   Hypertension                   81 non-null     float64
 8   Gender                         81 non-null     float64
 9   Diabetes                       81 non-null     float64
 10  PreviousMI                     81 non-null     float64
dtypes: float64(11)
memory usage: 7.1 KB


In [3]:
with pm.Model() as m:
    X_data = pm.Data("X_data", X_aug)
    y_data = pm.Data("y_data", y)

    betas = pm.Normal("beta", mu=0, tau=0.001, shape=X.shape[1] + 1)

    p = dot(X_data, betas)

    lik = pm.Bernoulli("y", logit_p=p, observed=y_data)

    trace = pm.sample(
        10000,
        chains=4,
        tune=500,
        cores=4,
        random_seed=1,
    )

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/pymc/data.py:406: UserWarning: The `mutable` kwarg was not specified. Before v4.1.0 it defaulted to `pm.Data(mutable=True)`, which is equivalent to using `pm.MutableData()`. In v4.1.0 the default changed to `pm.Data(mutable=False)`, equivalent to `pm.ConstantData`. Use `pm.ConstantData`/`pm.MutableData` or pass `pm.Data(..., mutable=False/True)` to avoid this warning.
  warnings.warn(
/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/multipledispatch/dispatcher.py:27: AmbiguityWarning: 
Ambiguities exist in dispatched function _unify

The following signatures may result in ambiguous behavior:
	[object, ConstrainedVar, Mapping], [ConstrainedVar, Var, Mapping]
	[ConstrainedVar, object, Mapping], [object, ConstrainedVar, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, object, Mapping]
	[ConstrainedVar, Var, Mapping], [object, ConstrainedVar, Mapping]


Consider making the following addition

Sampling 4 chains for 500 tune and 10_000 draw iterations (2_000 + 40_000 draws total) took 45 seconds.


In [4]:
az.summary(trace, hdi_prob=0.95)

mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
beta[0]  -13.048  4.964   -23.140     -3.746      0.042    0.030   14312.0   
beta[1]    0.188  0.050     0.095      0.287      0.000    0.000   21353.0   
beta[2]    0.033  0.026    -0.016      0.085      0.000    0.000   17598.0   
beta[3]   -0.023  0.016    -0.057      0.006      0.000    0.000   18693.0   
beta[4]   -0.155  0.096    -0.349      0.029      0.000    0.000   41345.0   
beta[5]    0.006  0.032    -0.055      0.070      0.000    0.000   21622.0   
beta[6]    0.025  0.028    -0.033      0.079      0.000    0.000   25561.0   
beta[7]   -0.652  0.667    -1.943      0.660      0.004    0.003   36306.0   
beta[8]   -0.323  0.690    -1.681      1.022      0.004    0.003   34897.0   
beta[9]    1.315  0.700    -0.018      2.719      0.004    0.003   25057.0   
beta[10]   0.404  0.702    -1.004      1.758      0.004    0.003   28424.0   

          ess_tail  r_hat  
beta[0]    19109.0    1.0  
beta[1]    21984.0    1.0  
beta[2]    22163.0    1.0  
beta[3]    23929.0    1.0  
beta[4]    29473.0    1.0  
beta[5]    25470.0    1.0  
beta[6]    27443.0    1.0  
beta[7]    28271.0    1.0  
beta[8]    27656.0    1.0  
beta[9]    26796.0    1.0  
beta[10]   28540.0    1.0

In [5]:
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Fri Feb 03 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

aesara: 2.8.10
aeppl : 0.1.1

pandas: 1.5.3
pymc  : 5.0.1
numpy : 1.24.1
arviz : 0.14.0

